# Library


In [2]:
import pandas as pd
import os
import pyexasol

# folder

In [3]:
raw1_fold = 'D:/data/big2/BD1/raw'
df1_fold = 'D:/data/big2/BD1/df'
raw3_fold = 'D:/data/big2/BD3/raw'
df3_fold = 'D:/data/big2/BD3/df'

# server

In [4]:
# # insider db
# wd = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='VSYSD')
# we = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='vsyse')

In [5]:
# exasol db
wd = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSD')
we = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='vsyse')
wbt = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='wbt')

# Load

## RH에서 제공한 법정동코드

In [5]:
name = 'tb_mapdata_gu(2023.04.20)'
file_name = f'{name}.csv'
df = pd.read_csv(os.path.join(raw1_fold, file_name), low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   dong_code    278 non-null    int64  
 1   ctprvn_code  278 non-null    int64  
 2   signgu_code  278 non-null    int64  
 3   sdstrd_code  278 non-null    int64  
 4   ctprvn_nm    278 non-null    object 
 5   signgu_nm    260 non-null    object 
 6   sdstrd_nm    0 non-null      float64
dtypes: float64(1), int64(4), object(2)
memory usage: 15.3+ KB


In [6]:
df.head()

,dong_code,ctprvn_code,signgu_code,sdstrd_code,ctprvn_nm,signgu_nm,sdstrd_nm
0,0,0,0,0,전국,NaN,NaN
1,1100000000,11,0,0,서울특별시,NaN,NaN
2,1111000000,11,110,0,서울특별시,종로구,NaN
3,1114000000,11,140,0,서울특별시,중구,NaN
4,1117000000,11,170,0,서울특별시,용산구,NaN


In [8]:
TB_MAPDATA = df.copy()

### [upload] TB_MAPDATA

In [9]:
expdf = TB_MAPDATA
table_nm = 'TB_MAPDATA'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 1s
we.import_from_pandas(expdf, table_nm)

## 4등급 result(GRD4_RESULT)

In [10]:
# 9.9s
name = '4등급_차량_리스트_양식_result_최종(전체)(2023.07.03)'
file_name = f'{name}.csv'
df = pd.read_csv(os.path.join(raw1_fold, file_name), low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1502235 entries, 0 to 1502234
Data columns (total 28 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   차대번호      1502235 non-null  object 
 1   차량번호      1502235 non-null  object 
 2   제원관리번호    1502235 non-null  object 
 3   차종        1502235 non-null  object 
 4   용도        1502235 non-null  object 
 5   최초등록일     1502235 non-null  int64  
 6   연식        1502235 non-null  int64  
 7   제작일자      1335178 non-null  float64
 8   검사유효일     1502223 non-null  float64
 9   차명        1502235 non-null  object 
 10  차종분류      1502235 non-null  object 
 11  차종유형      1502235 non-null  object 
 12  자동차형식     1501958 non-null  object 
 13  제작사명      1492590 non-null  object 
 14  연료        1502235 non-null  object 
 15  엔진형식      1500400 non-null  object 
 16  총중량       1502234 non-null  float64
 17  적재중량      1501728 non-null  float64
 18  엔진출력      1501882 non-null  object 
 19  배기량       1502231 non

In [11]:
df.head()

,차대번호,차량번호,제원관리번호,차종,용도,최초등록일,연식,제작일자,검사유효일,차명,...,엔진출력,배기량,법정동코드,시도명,시군구명,소유자구분,말소여부,배인번호_수정,등급_수정,DPF유무_수정
0,WAUZZZ4F16N115327,21로7811,02420000100031305,승용,자가용,20060208,2006,20060119.0,20200207.0,아우디 A6 3.0 TDI Quattro,...,225.0,2967.0,3017011600,대전광역시,서구,주민,N,4MY-AD-20,4.0,무
1,WAUZZZ4F16N113691,23거2347,02420000100031305,승용,자가용,20060228,2006,20060116.0,20220227.0,아우디 A6 3.0 TDI Quattro,...,225.0,2967.0,2635010700,부산광역시,해운대구,외국인,N,4MY-AD-20,4.0,무
2,WAUZZZ4F86N088157,04어9316,02420000100031305,승용,자가용,20060105,2006,20051212.0,20200104.0,아우디 A6 3.0 TDI Quattro,...,225.0,2967.0,4148012200,경기도,파주시,주민,N,4MY-AD-20,4.0,무
3,WAUZZZ4F36N090463,14우2622,02420000100031305,승용,자가용,20060105,2006,20051206.0,20100104.0,아우디 A6 3.0 TDI Quattro,...,225.0,2967.0,2623010900,부산광역시,부산진구,주민,N,4MY-AD-20,4.0,무
4,WAUZZZ4F26N152385,07머3269,02420000100031305,승용,자가용,20060630,2006,20060612.0,20220629.0,아우디 A6 3.0 TDI Quattro,...,225.0,2967.0,5013010300,제주특별자치도,서귀포시,주민,N,4MY-AD-20,4.0,무


In [12]:
GRD4_RESULT = df.copy()

### [upload] GRD4_RESULT

In [13]:
expdf = GRD4_RESULT
table_nm = 'GRD4_RESULT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 1s
we.import_from_pandas(expdf, table_nm)

# BD1

## STD_BD_GRD4_CAR_CURSTT

In [ ]:
STD_BD_GRD4_CAR_CURSTT = pd.read_csv(os.path.join(df1_fold, 'STD_BD_GRD4_CAR_CURSTT.csv'))
STD_BD_GRD4_CAR_CURSTT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162570 entries, 0 to 1162569
Data columns (total 15 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   LOAD_DT                        1162570 non-null  int64  
 1   VIN                            1162570 non-null  object 
 2   RGN                            1162570 non-null  object 
 3   CTPV                           1162570 non-null  object 
 4   DPF_EXTRNS_YN                  1162570 non-null  object 
 5   VHCTY_CD                       1162558 non-null  object 
 6   VHCTY_TY                       1162570 non-null  object 
 7   SEASON_1ODR_CRDN_NOCS          1162570 non-null  float64
 8   SEASON_2ODR_CRDN_NOCS          1162570 non-null  float64
 9   SEASON_3ODR_CRDN_NOCS          1162570 non-null  float64
 10  SEASON_4ODR_CRDN_NOCS          1162570 non-null  float64
 11  SEASON_1ODR_CRDN_NOCS_DY_AVRG  1162570 non-null  float64
 12  SEASON_2ODR_CR

In [ ]:
expdf = STD_BD_GRD4_CAR_CURSTT
table_nm = 'STD_BD_GRD4_CAR_CURSTT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 9s
we.import_from_pandas(expdf, table_nm)

## STD_BD_GRD4_SI

In [ ]:
STD_BD_GRD4_SI = pd.read_csv(os.path.join(df1_fold, 'STD_BD_GRD4_SI.csv'))
STD_BD_GRD4_SI.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162570 entries, 0 to 1162569
Data columns (total 15 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   LOAD_DT                        1162570 non-null  int64  
 1   VIN                            1162570 non-null  object 
 2   RGN                            1162570 non-null  object 
 3   CTPV                           1162570 non-null  object 
 4   DPF_EXTRNS_YN                  1162570 non-null  object 
 5   VHCTY_CD                       1162558 non-null  object 
 6   VHCTY_TY                       1162570 non-null  object 
 7   SEASON_1ODR_CRDN_NOCS          1162570 non-null  float64
 8   SEASON_2ODR_CRDN_NOCS          1162570 non-null  float64
 9   SEASON_3ODR_CRDN_NOCS          1162570 non-null  float64
 10  SEASON_4ODR_CRDN_NOCS          1162570 non-null  float64
 11  SEASON_1ODR_CRDN_NOCS_DY_AVRG  1162570 non-null  float64
 12  SEASON_2ODR_CR

In [ ]:
expdf = STD_BD_GRD4_SI
table_nm = 'STD_BD_GRD4_SI'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 6s
we.import_from_pandas(expdf, table_nm)

## STD_BD_GRD4_CAR_CURSTT_TOT

In [ ]:
STD_BD_GRD4_CAR_CURSTT_TOT = pd.read_csv(os.path.join(df1_fold, 'STD_BD_GRD4_CAR_CURSTT_TOT.csv'))
STD_BD_GRD4_CAR_CURSTT_TOT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162570 entries, 0 to 1162569
Data columns (total 15 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   LOAD_DT                        1162570 non-null  int64  
 1   VIN                            1162570 non-null  object 
 2   RGN                            1162570 non-null  object 
 3   CTPV                           1162570 non-null  object 
 4   DPF_EXTRNS_YN                  1162570 non-null  object 
 5   VHCTY_CD                       1162558 non-null  object 
 6   VHCTY_TY                       1162570 non-null  object 
 7   SEASON_1ODR_CRDN_NOCS          1162570 non-null  float64
 8   SEASON_2ODR_CRDN_NOCS          1162570 non-null  float64
 9   SEASON_3ODR_CRDN_NOCS          1162570 non-null  float64
 10  SEASON_4ODR_CRDN_NOCS          1162570 non-null  float64
 11  SEASON_1ODR_CRDN_NOCS_DY_AVRG  1162570 non-null  float64
 12  SEASON_2ODR_CR

In [ ]:
expdf = STD_BD_GRD4_CAR_CURSTT_TOT
table_nm = 'STD_BD_GRD4_CAR_CURSTT_TOT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 1s
we.import_from_pandas(expdf, table_nm)

## STD_BD_DAT_GRD4_CERT_NO_RVW

In [6]:
# 0s
STD_BD_DAT_GRD4_CERT_NO_RVW = pd.read_csv(os.path.join(df1_fold, 'STD_BD_DAT_GRD4_CERT_NO_RVW.csv'))
STD_BD_DAT_GRD4_CERT_NO_RVW.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1080 entries, 0 to 1079
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   EXHST_GAS_CERT_NO  1068 non-null   object 
 1   INSP_MTHD          1080 non-null   object 
 2   RVW_SE             1080 non-null   object 
 3   RPRS_MNFCTR_NM     1080 non-null   object 
 4   RPRS_VHCNM         1080 non-null   object 
 5   MANG_MNG_NO        1080 non-null   object 
 6   LOWR_QRT           1080 non-null   float64
 7   MID_QRT            1080 non-null   float64
 8   UP_QRT             1080 non-null   float64
 9   LOAD_DT            1080 non-null   int64  
dtypes: float64(3), int64(1), object(6)
memory usage: 84.5+ KB


In [7]:
expdf = STD_BD_DAT_GRD4_CERT_NO_RVW
table_nm = 'STD_BD_DAT_GRD4_CERT_NO_RVW'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 1s
we.import_from_pandas(expdf, table_nm)

## STD_BD_DAT_GRD4_SI

In [8]:
# 0s
STD_BD_DAT_GRD4_SI = pd.read_csv(os.path.join(df1_fold, 'STD_BD_DAT_GRD4_SI.csv'))
STD_BD_DAT_GRD4_SI.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405 entries, 0 to 404
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   EXHST_GAS_CERT_NO  401 non-null    object 
 1   INSP_MTHD          405 non-null    object 
 2   RPRS_MNFCTR_NM     405 non-null    object 
 3   RPRS_VHCNM         405 non-null    object 
 4   RPRS_VHCTY_CD      405 non-null    object 
 5   RPRS_FUEL          405 non-null    object 
 6   FRST_YRIDNW        405 non-null    int64  
 7   SI                 398 non-null    float64
 8   LOAD_DT            405 non-null    int64  
dtypes: float64(1), int64(2), object(6)
memory usage: 28.6+ KB


In [9]:
expdf = STD_BD_DAT_GRD4_SI
table_nm = 'STD_BD_DAT_GRD4_SI'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 1s
we.import_from_pandas(expdf, table_nm)

## STD_BD_GRD4_ELPDSRC_CURSTT

In [ ]:
STD_BD_GRD4_ELPDSRC_CURSTT = pd.read_csv(os.path.join(df1_fold, 'STD_BD_GRD4_ELPDSRC_CURSTT.csv'))
STD_BD_GRD4_ELPDSRC_CURSTT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162570 entries, 0 to 1162569
Data columns (total 15 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   LOAD_DT                        1162570 non-null  int64  
 1   VIN                            1162570 non-null  object 
 2   RGN                            1162570 non-null  object 
 3   CTPV                           1162570 non-null  object 
 4   DPF_EXTRNS_YN                  1162570 non-null  object 
 5   VHCTY_CD                       1162558 non-null  object 
 6   VHCTY_TY                       1162570 non-null  object 
 7   SEASON_1ODR_CRDN_NOCS          1162570 non-null  float64
 8   SEASON_2ODR_CRDN_NOCS          1162570 non-null  float64
 9   SEASON_3ODR_CRDN_NOCS          1162570 non-null  float64
 10  SEASON_4ODR_CRDN_NOCS          1162570 non-null  float64
 11  SEASON_1ODR_CRDN_NOCS_DY_AVRG  1162570 non-null  float64
 12  SEASON_2ODR_CR

In [ ]:
expdf = STD_BD_GRD4_ELPDSRC_CURSTT
table_nm = 'STD_BD_GRD4_ELPDSRC_CURSTT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 5s
we.import_from_pandas(expdf, table_nm)

## STD_BD_GRD4_MLSFC_RSLT

In [ ]:
STD_BD_GRD4_MLSFC_RSLT = pd.read_csv(os.path.join(df1_fold, 'STD_BD_GRD4_MLSFC_RSLT.csv'))
STD_BD_GRD4_MLSFC_RSLT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162570 entries, 0 to 1162569
Data columns (total 15 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   LOAD_DT                        1162570 non-null  int64  
 1   VIN                            1162570 non-null  object 
 2   RGN                            1162570 non-null  object 
 3   CTPV                           1162570 non-null  object 
 4   DPF_EXTRNS_YN                  1162570 non-null  object 
 5   VHCTY_CD                       1162558 non-null  object 
 6   VHCTY_TY                       1162570 non-null  object 
 7   SEASON_1ODR_CRDN_NOCS          1162570 non-null  float64
 8   SEASON_2ODR_CRDN_NOCS          1162570 non-null  float64
 9   SEASON_3ODR_CRDN_NOCS          1162570 non-null  float64
 10  SEASON_4ODR_CRDN_NOCS          1162570 non-null  float64
 11  SEASON_1ODR_CRDN_NOCS_DY_AVRG  1162570 non-null  float64
 12  SEASON_2ODR_CR

In [ ]:
expdf = STD_BD_GRD4_MLSFC_RSLT
table_nm = 'STD_BD_GRD4_MLSFC_RSLT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 7s
we.import_from_pandas(expdf, table_nm)

## STD_BD_DAT_GRD4_MLSFC

In [10]:
# 0s
STD_BD_DAT_GRD4_MLSFC = pd.read_csv(os.path.join(df1_fold, 'STD_BD_DAT_GRD4_MLSFC.csv'))
STD_BD_DAT_GRD4_MLSFC.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9122 entries, 0 to 9121
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   FUEL_CD    9122 non-null   object 
 1   CTPV       9122 non-null   object 
 2   SGG        9122 non-null   object 
 3   VHCTY_CD   9122 non-null   object 
 4   VHCTY_TY   9122 non-null   object 
 5   PURPS_CD2  9122 non-null   object 
 6   A_MKCNT    5575 non-null   float64
 7   B_MKCNT    3996 non-null   float64
 8   C_MKCNT    3574 non-null   float64
 9   D_MKCNT    861 non-null    float64
 10  X_MKCNT    4940 non-null   float64
 11  YR         9122 non-null   int64  
 12  LOAD_DT    9122 non-null   int64  
dtypes: float64(5), int64(2), object(6)
memory usage: 926.6+ KB


In [11]:
expdf = STD_BD_DAT_GRD4_MLSFC
table_nm = 'STD_BD_DAT_GRD4_MLSFC'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 1s
we.import_from_pandas(expdf, table_nm)

## STD_BD_DAT_GRD4_DTL_INFO

In [12]:
# 6s
STD_BD_DAT_GRD4_DTL_INFO = pd.read_csv(os.path.join(df1_fold, 'STD_BD_DAT_GRD4_DTL_INFO.csv'))
STD_BD_DAT_GRD4_DTL_INFO.info()

C:\Users\kbjung\AppData\Local\Temp\ipykernel_17788\1988223461.py:1: DtypeWarning: Columns (10,11,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  STD_BD_DAT_GRD4_DTL_INFO = pd.read_csv(os.path.join(df1_fold, 'STD_BD_DAT_GRD4_DTL_INFO.csv'))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1898763 entries, 0 to 1898762
Data columns (total 18 columns):
 #   Column           Dtype  
---  ------           -----  
 0   VHRNO            object 
 1   VIN              object 
 2   GRD4_MLSFC       object 
 3   VHCTY_CD         object 
 4   VHCTY_TY         object 
 5   PURPS_CD2        object 
 6   FUEL_CD          object 
 7   CTPV             object 
 8   SGG              object 
 9   YRIDNW           int64  
 10  DPF_MNTNG_YN     object 
 11  RDCDVC_KND       object 
 12  FRST_REG_YMD     int64  
 13  ELPDSRC_APLY_YN  object 
 14  ELPDSRC_STTS_CD  object 
 15  CHNG_DE          float64
 16  VHCL_ERSR_YN     object 
 17  LOAD_DT          int64  
dtypes: float64(1), int64(3), object(14)
memory usage: 260.8+ MB


In [13]:
expdf = STD_BD_DAT_GRD4_DTL_INFO
table_nm = 'STD_BD_DAT_GRD4_DTL_INFO'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 9s
we.import_from_pandas(expdf, table_nm)

## STD_BD_GRD4_RGN_CURSTT

In [ ]:
STD_BD_GRD4_RGN_CURSTT = pd.read_csv(os.path.join(df1_fold, 'STD_BD_GRD4_RGN_CURSTT.csv'))
STD_BD_GRD4_RGN_CURSTT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162570 entries, 0 to 1162569
Data columns (total 15 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   LOAD_DT                        1162570 non-null  int64  
 1   VIN                            1162570 non-null  object 
 2   RGN                            1162570 non-null  object 
 3   CTPV                           1162570 non-null  object 
 4   DPF_EXTRNS_YN                  1162570 non-null  object 
 5   VHCTY_CD                       1162558 non-null  object 
 6   VHCTY_TY                       1162570 non-null  object 
 7   SEASON_1ODR_CRDN_NOCS          1162570 non-null  float64
 8   SEASON_2ODR_CRDN_NOCS          1162570 non-null  float64
 9   SEASON_3ODR_CRDN_NOCS          1162570 non-null  float64
 10  SEASON_4ODR_CRDN_NOCS          1162570 non-null  float64
 11  SEASON_1ODR_CRDN_NOCS_DY_AVRG  1162570 non-null  float64
 12  SEASON_2ODR_CR

In [ ]:
expdf = STD_BD_GRD4_RGN_CURSTT
table_nm = 'STD_BD_GRD4_RGN_CURSTT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 5s
we.import_from_pandas(expdf, table_nm)

## STD_BD_GRD4_RGN_CURSTT_MOD

In [ ]:
STD_BD_GRD4_RGN_CURSTT_MOD = pd.read_csv(os.path.join(df1_fold, 'STD_BD_GRD4_RGN_CURSTT_MOD.csv'))
STD_BD_GRD4_RGN_CURSTT_MOD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162570 entries, 0 to 1162569
Data columns (total 15 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   LOAD_DT                        1162570 non-null  int64  
 1   VIN                            1162570 non-null  object 
 2   RGN                            1162570 non-null  object 
 3   CTPV                           1162570 non-null  object 
 4   DPF_EXTRNS_YN                  1162570 non-null  object 
 5   VHCTY_CD                       1162558 non-null  object 
 6   VHCTY_TY                       1162570 non-null  object 
 7   SEASON_1ODR_CRDN_NOCS          1162570 non-null  float64
 8   SEASON_2ODR_CRDN_NOCS          1162570 non-null  float64
 9   SEASON_3ODR_CRDN_NOCS          1162570 non-null  float64
 10  SEASON_4ODR_CRDN_NOCS          1162570 non-null  float64
 11  SEASON_1ODR_CRDN_NOCS_DY_AVRG  1162570 non-null  float64
 12  SEASON_2ODR_CR

In [ ]:
expdf = STD_BD_GRD4_RGN_CURSTT_MOD
table_nm = 'STD_BD_GRD4_RGN_CURSTT_MOD'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 5s
we.import_from_pandas(expdf, table_nm)

## STD_BD_DAT_GRD4_CAR_CURSTT

In [30]:
# 0s
STD_BD_DAT_GRD4_CAR_CURSTT = pd.read_csv(os.path.join(df1_fold, 'STD_BD_DAT_GRD4_CAR_CURSTT.csv'))
STD_BD_DAT_GRD4_CAR_CURSTT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9174 entries, 0 to 9173
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   YR                   9174 non-null   int64  
 1   MM                   9174 non-null   int64  
 2   CTPV                 9174 non-null   object 
 3   SGG                  9171 non-null   object 
 4   FUEL_CD              9127 non-null   object 
 5   VHCTY_CD             9172 non-null   object 
 6   VHCTY_TY             9174 non-null   object 
 7   PURPS_CD2            9174 non-null   object 
 8   VHCL_MKCNT           9174 non-null   int64  
 9   DEC_RT               0 non-null      float64
 10  RDCDVC_EXTRNS_MKCNT  9174 non-null   int64  
 11  RDCDVC_UNAT_MKCNT    9174 non-null   int64  
 12  ELPDSRC_MKCNT        9174 non-null   int64  
 13  LOAD_DT              9174 non-null   int64  
dtypes: float64(1), int64(7), object(6)
memory usage: 1003.5+ KB


In [31]:
expdf = STD_BD_DAT_GRD4_CAR_CURSTT
table_nm = 'STD_BD_DAT_GRD4_CAR_CURSTT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 1s
we.import_from_pandas(expdf, table_nm)

## STD_BD_GRD4_EXHST_GAS_MSS_TOT

In [ ]:
STD_BD_GRD4_EXHST_GAS_MSS_TOT = pd.read_csv(os.path.join(df1_fold, 'STD_BD_GRD4_EXHST_GAS_MSS_TOT.csv'))
STD_BD_GRD4_EXHST_GAS_MSS_TOT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162570 entries, 0 to 1162569
Data columns (total 15 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   LOAD_DT                        1162570 non-null  int64  
 1   VIN                            1162570 non-null  object 
 2   RGN                            1162570 non-null  object 
 3   CTPV                           1162570 non-null  object 
 4   DPF_EXTRNS_YN                  1162570 non-null  object 
 5   VHCTY_CD                       1162558 non-null  object 
 6   VHCTY_TY                       1162570 non-null  object 
 7   SEASON_1ODR_CRDN_NOCS          1162570 non-null  float64
 8   SEASON_2ODR_CRDN_NOCS          1162570 non-null  float64
 9   SEASON_3ODR_CRDN_NOCS          1162570 non-null  float64
 10  SEASON_4ODR_CRDN_NOCS          1162570 non-null  float64
 11  SEASON_1ODR_CRDN_NOCS_DY_AVRG  1162570 non-null  float64
 12  SEASON_2ODR_CR

In [ ]:
expdf = STD_BD_GRD4_EXHST_GAS_MSS_TOT
table_nm = 'STD_BD_GRD4_EXHST_GAS_MSS_TOT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 7s
we.import_from_pandas(expdf, table_nm)

## STD_BD_GRD4_EXHST_GAS_MSS

In [ ]:
STD_BD_GRD4_EXHST_GAS_MSS = pd.read_csv(os.path.join(df1_fold, 'STD_BD_GRD4_EXHST_GAS_MSS.csv'))
STD_BD_GRD4_EXHST_GAS_MSS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162570 entries, 0 to 1162569
Data columns (total 15 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   LOAD_DT                        1162570 non-null  int64  
 1   VIN                            1162570 non-null  object 
 2   RGN                            1162570 non-null  object 
 3   CTPV                           1162570 non-null  object 
 4   DPF_EXTRNS_YN                  1162570 non-null  object 
 5   VHCTY_CD                       1162558 non-null  object 
 6   VHCTY_TY                       1162570 non-null  object 
 7   SEASON_1ODR_CRDN_NOCS          1162570 non-null  float64
 8   SEASON_2ODR_CRDN_NOCS          1162570 non-null  float64
 9   SEASON_3ODR_CRDN_NOCS          1162570 non-null  float64
 10  SEASON_4ODR_CRDN_NOCS          1162570 non-null  float64
 11  SEASON_1ODR_CRDN_NOCS_DY_AVRG  1162570 non-null  float64
 12  SEASON_2ODR_CR

In [ ]:
expdf = STD_BD_GRD4_EXHST_GAS_MSS
table_nm = 'STD_BD_GRD4_EXHST_GAS_MSS'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 30.7s
we.import_from_pandas(expdf, table_nm)

## STD_BD_DAT_GRD4_EXHST_MSS_CURSTT

In [14]:
# 0s
STD_BD_DAT_GRD4_EXHST_MSS_CURSTT = pd.read_csv(os.path.join(df1_fold, 'STD_BD_DAT_GRD4_EXHST_MSS_CURSTT.csv'))
STD_BD_DAT_GRD4_EXHST_MSS_CURSTT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9092 entries, 0 to 9091
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   YR                  9092 non-null   int64  
 1   CTPV                9092 non-null   object 
 2   SGG                 9092 non-null   object 
 3   FUEL_CD             9092 non-null   object 
 4   VHCTY_CD            9092 non-null   object 
 5   VHCTY_TY            9092 non-null   object 
 6   PURPS_CD2           9092 non-null   object 
 7   CO_EXHST_MSS_SM     9092 non-null   float64
 8   HC_EXHST_MSS_SM     9092 non-null   float64
 9   NOx_EXHST_MSS_SM    9092 non-null   float64
 10  PM10_EXHST_MSS_SM   9092 non-null   float64
 11  PM2_5_EXHST_MSS_SM  9092 non-null   float64
 12  LOAD_DT             9092 non-null   int64  
dtypes: float64(5), int64(2), object(6)
memory usage: 923.5+ KB


In [15]:
expdf = STD_BD_DAT_GRD4_EXHST_MSS_CURSTT
table_nm = 'STD_BD_DAT_GRD4_EXHST_MSS_CURSTT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 1s
we.import_from_pandas(expdf, table_nm)

## STD_BD_DAT_GRD4_MEVLU

In [16]:
# 0s
STD_BD_DAT_GRD4_MEVLU = pd.read_csv(os.path.join(df1_fold, 'STD_BD_DAT_GRD4_MEVLU.csv'))
STD_BD_DAT_GRD4_MEVLU.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9092 entries, 0 to 9091
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   YR                    9092 non-null   int64  
 1   CTPV                  9092 non-null   object 
 2   SGG                   9092 non-null   object 
 3   FUEL_CD               9092 non-null   object 
 4   VHCTY_CD              9092 non-null   object 
 5   VHCTY_TY              9092 non-null   object 
 6   PURPS_CD2             9092 non-null   object 
 7   CO_EXHST_MSS_AVRG     9092 non-null   float64
 8   HC_EXHST_MSS_AVRG     9092 non-null   float64
 9   NOx_EXHST_MSS_AVRG    9092 non-null   float64
 10  PM10_EXHST_MSS_AVRG   9092 non-null   float64
 11  PM2_5_EXHST_MSS_AVRG  9092 non-null   float64
 12  LOAD_DT               9092 non-null   int64  
dtypes: float64(5), int64(2), object(6)
memory usage: 923.5+ KB


In [17]:
expdf = STD_BD_DAT_GRD4_MEVLU
table_nm = 'STD_BD_DAT_GRD4_MEVLU'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 1s
we.import_from_pandas(expdf, table_nm)

# BD3

## STD_BD_CAR_CURSTT_MOD

In [ ]:
STD_BD_CAR_CURSTT_MOD = pd.read_csv(os.path.join(df3_fold, 'STD_BD_CAR_CURSTT_MOD.csv'))
STD_BD_CAR_CURSTT_MOD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162570 entries, 0 to 1162569
Data columns (total 15 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   LOAD_DT                        1162570 non-null  int64  
 1   VIN                            1162570 non-null  object 
 2   RGN                            1162570 non-null  object 
 3   CTPV                           1162570 non-null  object 
 4   DPF_EXTRNS_YN                  1162570 non-null  object 
 5   VHCTY_CD                       1162558 non-null  object 
 6   VHCTY_TY                       1162570 non-null  object 
 7   SEASON_1ODR_CRDN_NOCS          1162570 non-null  float64
 8   SEASON_2ODR_CRDN_NOCS          1162570 non-null  float64
 9   SEASON_3ODR_CRDN_NOCS          1162570 non-null  float64
 10  SEASON_4ODR_CRDN_NOCS          1162570 non-null  float64
 11  SEASON_1ODR_CRDN_NOCS_DY_AVRG  1162570 non-null  float64
 12  SEASON_2ODR_CR

In [ ]:
expdf = STD_BD_CAR_CURSTT_MOD
table_nm = 'STD_BD_CAR_CURSTT_MOD'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 5s
we.import_from_pandas(expdf, table_nm)

## STD_BD_CAR_CURSTT_MOD2

In [ ]:
STD_BD_CAR_CURSTT_MOD2 = pd.read_csv(os.path.join(df3_fold, 'STD_BD_CAR_CURSTT_MOD2.csv'))
STD_BD_CAR_CURSTT_MOD2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162570 entries, 0 to 1162569
Data columns (total 15 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   LOAD_DT                        1162570 non-null  int64  
 1   VIN                            1162570 non-null  object 
 2   RGN                            1162570 non-null  object 
 3   CTPV                           1162570 non-null  object 
 4   DPF_EXTRNS_YN                  1162570 non-null  object 
 5   VHCTY_CD                       1162558 non-null  object 
 6   VHCTY_TY                       1162570 non-null  object 
 7   SEASON_1ODR_CRDN_NOCS          1162570 non-null  float64
 8   SEASON_2ODR_CRDN_NOCS          1162570 non-null  float64
 9   SEASON_3ODR_CRDN_NOCS          1162570 non-null  float64
 10  SEASON_4ODR_CRDN_NOCS          1162570 non-null  float64
 11  SEASON_1ODR_CRDN_NOCS_DY_AVRG  1162570 non-null  float64
 12  SEASON_2ODR_CR

In [ ]:
expdf = STD_BD_CAR_CURSTT_MOD2
table_nm = 'STD_BD_CAR_CURSTT_MOD2'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 1s
we.import_from_pandas(expdf, table_nm)

## STD_BD_CAR_CURSTT

In [ ]:
STD_BD_CAR_CURSTT = pd.read_csv(os.path.join(df3_fold, 'STD_BD_CAR_CURSTT.csv'))
STD_BD_CAR_CURSTT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162570 entries, 0 to 1162569
Data columns (total 15 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   LOAD_DT                        1162570 non-null  int64  
 1   VIN                            1162570 non-null  object 
 2   RGN                            1162570 non-null  object 
 3   CTPV                           1162570 non-null  object 
 4   DPF_EXTRNS_YN                  1162570 non-null  object 
 5   VHCTY_CD                       1162558 non-null  object 
 6   VHCTY_TY                       1162570 non-null  object 
 7   SEASON_1ODR_CRDN_NOCS          1162570 non-null  float64
 8   SEASON_2ODR_CRDN_NOCS          1162570 non-null  float64
 9   SEASON_3ODR_CRDN_NOCS          1162570 non-null  float64
 10  SEASON_4ODR_CRDN_NOCS          1162570 non-null  float64
 11  SEASON_1ODR_CRDN_NOCS_DY_AVRG  1162570 non-null  float64
 12  SEASON_2ODR_CR

In [ ]:
expdf = STD_BD_CAR_CURSTT
table_nm = 'STD_BD_CAR_CURSTT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 1m 56.2s
we.import_from_pandas(expdf, table_nm)

## STD_BD_CAR_REG_MKCNT

In [ ]:
STD_BD_CAR_REG_MKCNT = pd.read_csv(os.path.join(df3_fold, 'STD_BD_CAR_REG_MKCNT.csv'))
STD_BD_CAR_REG_MKCNT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162570 entries, 0 to 1162569
Data columns (total 15 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   LOAD_DT                        1162570 non-null  int64  
 1   VIN                            1162570 non-null  object 
 2   RGN                            1162570 non-null  object 
 3   CTPV                           1162570 non-null  object 
 4   DPF_EXTRNS_YN                  1162570 non-null  object 
 5   VHCTY_CD                       1162558 non-null  object 
 6   VHCTY_TY                       1162570 non-null  object 
 7   SEASON_1ODR_CRDN_NOCS          1162570 non-null  float64
 8   SEASON_2ODR_CRDN_NOCS          1162570 non-null  float64
 9   SEASON_3ODR_CRDN_NOCS          1162570 non-null  float64
 10  SEASON_4ODR_CRDN_NOCS          1162570 non-null  float64
 11  SEASON_1ODR_CRDN_NOCS_DY_AVRG  1162570 non-null  float64
 12  SEASON_2ODR_CR

In [ ]:
expdf = STD_BD_CAR_REG_MKCNT
table_nm = 'STD_BD_CAR_REG_MKCNT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 7s
we.import_from_pandas(expdf, table_nm)

## STD_BD_CAR_PRET

In [10]:
STD_BD_CAR_PRET = pd.read_csv(os.path.join(df3_fold, 'STD_BD_CAR_PRET.csv'))
STD_BD_CAR_PRET.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   LOAD_DT         17 non-null     int64  
 1   YR              17 non-null     int64  
 2   DSL             4 non-null      float64
 3   GSL             4 non-null      float64
 4   LPG             4 non-null      float64
 5   DSL_PRET        13 non-null     float64
 6   DSL_PRET_BSPLN  13 non-null     float64
 7   DSL_PRET_AKM    13 non-null     float64
 8   GSL_PRET        13 non-null     float64
 9   GSL_PRET_BSPLN  13 non-null     float64
 10  GSL_PRET_AKM    13 non-null     float64
 11  LPG_PRET        13 non-null     float64
 12  LPG_PRET_BSPLN  13 non-null     float64
 13  LPG_PRET_AKM    13 non-null     float64
dtypes: float64(12), int64(2)
memory usage: 2.0 KB


In [11]:
expdf = STD_BD_CAR_PRET
table_nm = 'STD_BD_CAR_PRET'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 7s
we.import_from_pandas(expdf, table_nm)

## STD_BD_HYBRD_CAR_PRET

In [12]:
STD_BD_HYBRD_CAR_PRET = pd.read_csv(os.path.join(df3_fold, 'STD_BD_HYBRD_CAR_PRET.csv'))
STD_BD_HYBRD_CAR_PRET.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   LOAD_DT    17 non-null     int64  
 1   YR         17 non-null     int64  
 2   GSLH       4 non-null      float64
 3   DSLH       4 non-null      float64
 4   LPGH       4 non-null      float64
 5   GSLH_PRET  13 non-null     float64
 6   DSLH_PRET  13 non-null     float64
 7   LPGH_PRET  13 non-null     float64
dtypes: float64(6), int64(2)
memory usage: 1.2 KB


In [13]:
expdf = STD_BD_HYBRD_CAR_PRET
table_nm = 'STD_BD_HYBRD_CAR_PRET'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 7s
we.import_from_pandas(expdf, table_nm)

## STD_BD_FUEL_GRD_VHCL_CURSTT_PRET

In [ ]:
STD_BD_FUEL_GRD_VHCL_CURSTT_PRET = pd.read_csv(os.path.join(df3_fold, 'STD_BD_FUEL_GRD_VHCL_CURSTT_PRET.csv'))
STD_BD_FUEL_GRD_VHCL_CURSTT_PRET.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162570 entries, 0 to 1162569
Data columns (total 15 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   LOAD_DT                        1162570 non-null  int64  
 1   VIN                            1162570 non-null  object 
 2   RGN                            1162570 non-null  object 
 3   CTPV                           1162570 non-null  object 
 4   DPF_EXTRNS_YN                  1162570 non-null  object 
 5   VHCTY_CD                       1162558 non-null  object 
 6   VHCTY_TY                       1162570 non-null  object 
 7   SEASON_1ODR_CRDN_NOCS          1162570 non-null  float64
 8   SEASON_2ODR_CRDN_NOCS          1162570 non-null  float64
 9   SEASON_3ODR_CRDN_NOCS          1162570 non-null  float64
 10  SEASON_4ODR_CRDN_NOCS          1162570 non-null  float64
 11  SEASON_1ODR_CRDN_NOCS_DY_AVRG  1162570 non-null  float64
 12  SEASON_2ODR_CR

In [ ]:
expdf = STD_BD_FUEL_GRD_VHCL_CURSTT_PRET
table_nm = 'STD_BD_FUEL_GRD_VHCL_CURSTT_PRET'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 7s
we.import_from_pandas(expdf, table_nm)

## STD_BD_ECO_CAR_PRET

In [ ]:
STD_BD_ECO_CAR_PRET = pd.read_csv(os.path.join(df3_fold, 'STD_BD_ECO_CAR_PRET.csv'))
STD_BD_ECO_CAR_PRET.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162570 entries, 0 to 1162569
Data columns (total 15 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   LOAD_DT                        1162570 non-null  int64  
 1   VIN                            1162570 non-null  object 
 2   RGN                            1162570 non-null  object 
 3   CTPV                           1162570 non-null  object 
 4   DPF_EXTRNS_YN                  1162570 non-null  object 
 5   VHCTY_CD                       1162558 non-null  object 
 6   VHCTY_TY                       1162570 non-null  object 
 7   SEASON_1ODR_CRDN_NOCS          1162570 non-null  float64
 8   SEASON_2ODR_CRDN_NOCS          1162570 non-null  float64
 9   SEASON_3ODR_CRDN_NOCS          1162570 non-null  float64
 10  SEASON_4ODR_CRDN_NOCS          1162570 non-null  float64
 11  SEASON_1ODR_CRDN_NOCS_DY_AVRG  1162570 non-null  float64
 12  SEASON_2ODR_CR

In [ ]:
expdf = STD_BD_ECO_CAR_PRET
table_nm = 'STD_BD_ECO_CAR_PRET'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 7s
we.import_from_pandas(expdf, table_nm)

## STD_BD_ERSR_RSLT

In [ ]:
STD_BD_ERSR_RSLT = pd.read_csv(os.path.join(df3_fold, 'STD_BD_ERSR_RSLT.csv'))
STD_BD_ERSR_RSLT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162570 entries, 0 to 1162569
Data columns (total 15 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   LOAD_DT                        1162570 non-null  int64  
 1   VIN                            1162570 non-null  object 
 2   RGN                            1162570 non-null  object 
 3   CTPV                           1162570 non-null  object 
 4   DPF_EXTRNS_YN                  1162570 non-null  object 
 5   VHCTY_CD                       1162558 non-null  object 
 6   VHCTY_TY                       1162570 non-null  object 
 7   SEASON_1ODR_CRDN_NOCS          1162570 non-null  float64
 8   SEASON_2ODR_CRDN_NOCS          1162570 non-null  float64
 9   SEASON_3ODR_CRDN_NOCS          1162570 non-null  float64
 10  SEASON_4ODR_CRDN_NOCS          1162570 non-null  float64
 11  SEASON_1ODR_CRDN_NOCS_DY_AVRG  1162570 non-null  float64
 12  SEASON_2ODR_CR

In [ ]:
expdf = STD_BD_ERSR_RSLT
table_nm = 'STD_BD_ERSR_RSLT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 12.6s
we.import_from_pandas(expdf, table_nm)

## STD_BD_DAT_GRD_CURSTT

In [18]:
# 1s
STD_BD_DAT_GRD_CURSTT = pd.read_csv(os.path.join(df3_fold, 'STD_BD_DAT_GRD_CURSTT.csv'))
STD_BD_DAT_GRD_CURSTT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293760 entries, 0 to 293759
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   CTPV              293760 non-null  object 
 1   YR                293760 non-null  int64  
 2   MM                293760 non-null  int64  
 3   EXHST_GAS_GRD_CD  293760 non-null  object 
 4   FUEL_CD           293760 non-null  object 
 5   VHCTY_CD          293760 non-null  object 
 6   VHCTY_TY          293760 non-null  object 
 7   PURPS_CD2         293760 non-null  object 
 8   VHCL_MKCNT        293760 non-null  float64
 9   ERSR_MKCNT        293760 non-null  float64
 10  YR_FUEL_VHCL_RT   293760 non-null  float64
 11  LOAD_DT           293760 non-null  int64  
dtypes: float64(3), int64(3), object(6)
memory usage: 26.9+ MB


In [19]:
expdf = STD_BD_DAT_GRD_CURSTT
table_nm = 'STD_BD_DAT_GRD_CURSTT'.upper()

# 테이블 생성
try:
    sql = 'create or replace table ' + table_nm + '( \n'

    for idx,column in enumerate(expdf.columns):
        if 'float' in expdf[column].dtype.name:
            sql += column + ' float'
        elif 'int' in expdf[column].dtype.name:
            sql += column + ' number'
        else:
            sql += column + ' varchar(255)'

        if len(expdf.columns) - 1 != idx:
            sql += ','
        sql += '\n'
    sql += ')'    
    we.execute(sql)
    we.import_from_pandas(expdf, table_nm)
except:
    # 데이터 추가
    # 2s
    we.import_from_pandas(expdf, table_nm)

## STD_BD_DAT_FUEL_CAR_DEC

In [20]:
# 0s
STD_BD_DAT_FUEL_CAR_DEC = pd.read_csv(os.path.join(df3_fold, 'STD_BD_DAT_FUEL_CAR_DEC.csv'))
STD_BD_DAT_FUEL_CAR_DEC.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1224 entries, 0 to 1223
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   YR                1224 non-null   int64  
 1   QRT               1224 non-null   int64  
 2   EXHST_GAS_GRD_CD  1224 non-null   object 
 3   FUEL_CD           1224 non-null   object 
 4   LPG_MKCNT         96 non-null     float64
 5   LPG_PRET          312 non-null    float64
 6   LPG_PRET_BSPLN    312 non-null    float64
 7   LPG_PRET_AKM      312 non-null    float64
 8   DSL_MKCNT         96 non-null     float64
 9   DSL_PRET          312 non-null    float64
 10  DSL_PRET_BSPLN    312 non-null    float64
 11  DSL_PRET_AKM      312 non-null    float64
 12  GSL_MKCNT         96 non-null     float64
 13  GSL_PRET          312 non-null    float64
 14  GSL_PRET_BSPLN    312 non-null    float64
 15  GSL_PRET_AKM      312 non-null    float64
 16  LOAD_DT           1224 non-null   int64  


In [32]:
expdf = STD_BD_DAT_FUEL_CAR_DEC
table_nm = 'STD_BD_DAT_FUEL_CAR_DEC'.upper()

# 테이블 생성
try:
    sql = 'create or replace table ' + table_nm + '( \n'

    for idx,column in enumerate(expdf.columns):
        if 'float' in expdf[column].dtype.name:
            sql += column + ' float'
        elif 'int' in expdf[column].dtype.name:
            sql += column + ' number'
        else:
            sql += column + ' varchar(255)'

        if len(expdf.columns) - 1 != idx:
            sql += ','
        sql += '\n'
    sql += ')'    
    we.execute(sql)
    we.import_from_pandas(expdf, table_nm)
except:
    # 데이터 추가
    # 1s
    we.import_from_pandas(expdf, table_nm)

## STD_BD_GRD5_ELPDSRC_CURSTT

In [ ]:
STD_BD_GRD5_ELPDSRC_CURSTT = pd.read_csv(os.path.join(df3_fold, 'STD_BD_GRD5_ELPDSRC_CURSTT.csv'))
STD_BD_GRD5_ELPDSRC_CURSTT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162570 entries, 0 to 1162569
Data columns (total 15 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   LOAD_DT                        1162570 non-null  int64  
 1   VIN                            1162570 non-null  object 
 2   RGN                            1162570 non-null  object 
 3   CTPV                           1162570 non-null  object 
 4   DPF_EXTRNS_YN                  1162570 non-null  object 
 5   VHCTY_CD                       1162558 non-null  object 
 6   VHCTY_TY                       1162570 non-null  object 
 7   SEASON_1ODR_CRDN_NOCS          1162570 non-null  float64
 8   SEASON_2ODR_CRDN_NOCS          1162570 non-null  float64
 9   SEASON_3ODR_CRDN_NOCS          1162570 non-null  float64
 10  SEASON_4ODR_CRDN_NOCS          1162570 non-null  float64
 11  SEASON_1ODR_CRDN_NOCS_DY_AVRG  1162570 non-null  float64
 12  SEASON_2ODR_CR

In [ ]:
expdf = STD_BD_GRD5_ELPDSRC_CURSTT
table_nm = 'STD_BD_GRD5_ELPDSRC_CURSTT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 5s
we.import_from_pandas(expdf, table_nm)

## STD_BD_GRD5_LEM_N_MOD

In [ ]:
STD_BD_GRD5_LEM_N_MOD = pd.read_csv(os.path.join(df3_fold, 'STD_BD_GRD5_LEM_N_MOD.csv'))
STD_BD_GRD5_LEM_N_MOD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162570 entries, 0 to 1162569
Data columns (total 15 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   LOAD_DT                        1162570 non-null  int64  
 1   VIN                            1162570 non-null  object 
 2   RGN                            1162570 non-null  object 
 3   CTPV                           1162570 non-null  object 
 4   DPF_EXTRNS_YN                  1162570 non-null  object 
 5   VHCTY_CD                       1162558 non-null  object 
 6   VHCTY_TY                       1162570 non-null  object 
 7   SEASON_1ODR_CRDN_NOCS          1162570 non-null  float64
 8   SEASON_2ODR_CRDN_NOCS          1162570 non-null  float64
 9   SEASON_3ODR_CRDN_NOCS          1162570 non-null  float64
 10  SEASON_4ODR_CRDN_NOCS          1162570 non-null  float64
 11  SEASON_1ODR_CRDN_NOCS_DY_AVRG  1162570 non-null  float64
 12  SEASON_2ODR_CR

In [ ]:
expdf = STD_BD_GRD5_LEM_N_MOD
table_nm = 'STD_BD_GRD5_LEM_N_MOD'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 5s
we.import_from_pandas(expdf, table_nm)

## STD_BD_SEASON_CRDN_NOCS_CURSTT

In [ ]:
STD_BD_SEASON_CRDN_NOCS_CURSTT = pd.read_csv(os.path.join(df3_fold, 'STD_BD_SEASON_CRDN_NOCS_CURSTT.csv'))
STD_BD_SEASON_CRDN_NOCS_CURSTT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162570 entries, 0 to 1162569
Data columns (total 15 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   LOAD_DT                        1162570 non-null  int64  
 1   VIN                            1162570 non-null  object 
 2   RGN                            1162570 non-null  object 
 3   CTPV                           1162570 non-null  object 
 4   DPF_EXTRNS_YN                  1162570 non-null  object 
 5   VHCTY_CD                       1162558 non-null  object 
 6   VHCTY_TY                       1162570 non-null  object 
 7   SEASON_1ODR_CRDN_NOCS          1162570 non-null  float64
 8   SEASON_2ODR_CRDN_NOCS          1162570 non-null  float64
 9   SEASON_3ODR_CRDN_NOCS          1162570 non-null  float64
 10  SEASON_4ODR_CRDN_NOCS          1162570 non-null  float64
 11  SEASON_1ODR_CRDN_NOCS_DY_AVRG  1162570 non-null  float64
 12  SEASON_2ODR_CR

In [ ]:
expdf = STD_BD_SEASON_CRDN_NOCS_CURSTT
table_nm = 'STD_BD_SEASON_CRDN_NOCS_CURSTT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 5s
we.import_from_pandas(expdf, table_nm)

## STD_BD_SEASON_DY_AVRG_CRDN_NOCS

In [ ]:
STD_BD_SEASON_DY_AVRG_CRDN_NOCS = pd.read_csv(os.path.join(df3_fold, 'STD_BD_SEASON_DY_AVRG_CRDN_NOCS.csv'))
STD_BD_SEASON_DY_AVRG_CRDN_NOCS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162570 entries, 0 to 1162569
Data columns (total 15 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   LOAD_DT                        1162570 non-null  int64  
 1   VIN                            1162570 non-null  object 
 2   RGN                            1162570 non-null  object 
 3   CTPV                           1162570 non-null  object 
 4   DPF_EXTRNS_YN                  1162570 non-null  object 
 5   VHCTY_CD                       1162558 non-null  object 
 6   VHCTY_TY                       1162570 non-null  object 
 7   SEASON_1ODR_CRDN_NOCS          1162570 non-null  float64
 8   SEASON_2ODR_CRDN_NOCS          1162570 non-null  float64
 9   SEASON_3ODR_CRDN_NOCS          1162570 non-null  float64
 10  SEASON_4ODR_CRDN_NOCS          1162570 non-null  float64
 11  SEASON_1ODR_CRDN_NOCS_DY_AVRG  1162570 non-null  float64
 12  SEASON_2ODR_CR

In [ ]:
expdf = STD_BD_SEASON_DY_AVRG_CRDN_NOCS
table_nm = 'STD_BD_SEASON_DY_AVRG_CRDN_NOCS'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 5s
we.import_from_pandas(expdf, table_nm)

## STD_BD_ORDITM_DSCL_CURSTT

In [ ]:
STD_BD_ORDITM_DSCL_CURSTT = pd.read_csv(os.path.join(df3_fold, 'STD_BD_ORDITM_DSCL_CURSTT.csv'))
STD_BD_ORDITM_DSCL_CURSTT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162570 entries, 0 to 1162569
Data columns (total 15 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   LOAD_DT                        1162570 non-null  int64  
 1   VIN                            1162570 non-null  object 
 2   RGN                            1162570 non-null  object 
 3   CTPV                           1162570 non-null  object 
 4   DPF_EXTRNS_YN                  1162570 non-null  object 
 5   VHCTY_CD                       1162558 non-null  object 
 6   VHCTY_TY                       1162570 non-null  object 
 7   SEASON_1ODR_CRDN_NOCS          1162570 non-null  float64
 8   SEASON_2ODR_CRDN_NOCS          1162570 non-null  float64
 9   SEASON_3ODR_CRDN_NOCS          1162570 non-null  float64
 10  SEASON_4ODR_CRDN_NOCS          1162570 non-null  float64
 11  SEASON_1ODR_CRDN_NOCS_DY_AVRG  1162570 non-null  float64
 12  SEASON_2ODR_CR

In [ ]:
expdf = STD_BD_ORDITM_DSCL_CURSTT
table_nm = 'STD_BD_ORDITM_DSCL_CURSTT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 5s
we.import_from_pandas(expdf, table_nm)

## STD_BD_SEASON_DSCL_RGN_CURSTT

In [ ]:
STD_BD_SEASON_DSCL_RGN_CURSTT = pd.read_csv(os.path.join(df3_fold, 'STD_BD_SEASON_DSCL_RGN_CURSTT.csv'))
STD_BD_SEASON_DSCL_RGN_CURSTT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162570 entries, 0 to 1162569
Data columns (total 15 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   LOAD_DT                        1162570 non-null  int64  
 1   VIN                            1162570 non-null  object 
 2   RGN                            1162570 non-null  object 
 3   CTPV                           1162570 non-null  object 
 4   DPF_EXTRNS_YN                  1162570 non-null  object 
 5   VHCTY_CD                       1162558 non-null  object 
 6   VHCTY_TY                       1162570 non-null  object 
 7   SEASON_1ODR_CRDN_NOCS          1162570 non-null  float64
 8   SEASON_2ODR_CRDN_NOCS          1162570 non-null  float64
 9   SEASON_3ODR_CRDN_NOCS          1162570 non-null  float64
 10  SEASON_4ODR_CRDN_NOCS          1162570 non-null  float64
 11  SEASON_1ODR_CRDN_NOCS_DY_AVRG  1162570 non-null  float64
 12  SEASON_2ODR_CR

In [ ]:
expdf = STD_BD_SEASON_DSCL_RGN_CURSTT
table_nm = 'STD_BD_SEASON_DSCL_RGN_CURSTT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 5s
we.import_from_pandas(expdf, table_nm)

## STD_BD_DAT_GRD5_REDUC_BIZ

In [22]:
# 0s
STD_BD_DAT_GRD5_REDUC_BIZ = pd.read_csv(os.path.join(df3_fold, 'STD_BD_DAT_GRD5_REDUC_BIZ.csv'))
STD_BD_DAT_GRD5_REDUC_BIZ.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   YR               4 non-null      int64  
 1   VHCL_MKCNT       4 non-null      float64
 2   NTRL_DCLN        3 non-null      float64
 3   ELPDSRC          4 non-null      int64  
 4   RDCDVC_1KND      4 non-null      float64
 5   RDCDVC_1KND_SCR  4 non-null      float64
 6   UNMNTNG          4 non-null      float64
 7   LOAD_DT          4 non-null      int64  
dtypes: float64(5), int64(3)
memory usage: 384.0 bytes


In [23]:
expdf = STD_BD_DAT_GRD5_REDUC_BIZ
table_nm = 'STD_BD_DAT_GRD5_REDUC_BIZ'.upper()

# 테이블 생성
try:
    sql = 'create table ' + table_nm + '( \n'

    for idx,column in enumerate(expdf.columns):
        if 'float' in expdf[column].dtype.name:
            sql += column + ' float'
        elif 'int' in expdf[column].dtype.name:
            sql += column + ' number'
        else:
            sql += column + ' varchar(255)'

        if len(expdf.columns) - 1 != idx:
            sql += ','
        sql += '\n'
    sql += ')'    
    we.execute(sql)
    we.import_from_pandas(expdf, table_nm)
except:
    # 데이터 추가
    # 1s
    we.import_from_pandas(expdf, table_nm)

## STD_BD_DAT_RUN_LMT_CURSTT

In [24]:
# 0s
STD_BD_DAT_RUN_LMT_CURSTT = pd.read_csv(os.path.join(df3_fold, 'STD_BD_DAT_RUN_LMT_CURSTT.csv'))
STD_BD_DAT_RUN_LMT_CURSTT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498 entries, 0 to 497
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   SEASON           498 non-null    object 
 1   RGN              498 non-null    object 
 2   CTPV             498 non-null    object 
 3   VHCTY_CD         498 non-null    object 
 4   VHCTY_TY         498 non-null    object 
 5   DSCL_VHCL_MKCNT  498 non-null    int64  
 6   UNLEM            498 non-null    int64  
 7   DUP_DSCL_MKCNT   425 non-null    float64
 8   LOAD_DT          498 non-null    int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 35.1+ KB


In [25]:
expdf = STD_BD_DAT_RUN_LMT_CURSTT
table_nm = 'STD_BD_DAT_RUN_LMT_CURSTT'.upper()

# 테이블 생성
try:
    sql = 'create table ' + table_nm + '( \n'

    for idx,column in enumerate(expdf.columns):
        if 'float' in expdf[column].dtype.name:
            sql += column + ' float'
        elif 'int' in expdf[column].dtype.name:
            sql += column + ' number'
        else:
            sql += column + ' varchar(255)'

        if len(expdf.columns) - 1 != idx:
            sql += ','
        sql += '\n'
    sql += ')'    
    we.execute(sql)
    we.import_from_pandas(expdf, table_nm)
except:
    # 데이터 추가
    # 1s
    we.import_from_pandas(expdf, table_nm)

## STD_BD_GRD4_DS_CRRLTN_CFFCNT

In [ ]:
STD_BD_GRD4_DS_CRRLTN_CFFCNT = pd.read_csv(os.path.join(df3_fold, 'STD_BD_GRD4_DS_CRRLTN_CFFCNT.csv'))
STD_BD_GRD4_DS_CRRLTN_CFFCNT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162570 entries, 0 to 1162569
Data columns (total 15 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   LOAD_DT                        1162570 non-null  int64  
 1   VIN                            1162570 non-null  object 
 2   RGN                            1162570 non-null  object 
 3   CTPV                           1162570 non-null  object 
 4   DPF_EXTRNS_YN                  1162570 non-null  object 
 5   VHCTY_CD                       1162558 non-null  object 
 6   VHCTY_TY                       1162570 non-null  object 
 7   SEASON_1ODR_CRDN_NOCS          1162570 non-null  float64
 8   SEASON_2ODR_CRDN_NOCS          1162570 non-null  float64
 9   SEASON_3ODR_CRDN_NOCS          1162570 non-null  float64
 10  SEASON_4ODR_CRDN_NOCS          1162570 non-null  float64
 11  SEASON_1ODR_CRDN_NOCS_DY_AVRG  1162570 non-null  float64
 12  SEASON_2ODR_CR

In [ ]:
expdf = STD_BD_GRD4_DS_CRRLTN_CFFCNT
table_nm = 'STD_BD_GRD4_DS_CRRLTN_CFFCNT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 5s
we.import_from_pandas(expdf, table_nm)

## STD_BD_GRD4_LEM_PRIO_ORD_SELCT_CURSTT

In [ ]:
STD_BD_GRD4_LEM_PRIO_ORD_SELCT_CURSTT = pd.read_csv(os.path.join(df3_fold, 'STD_BD_GRD4_LEM_PRIO_ORD_SELCT_CURSTT.csv'))
STD_BD_GRD4_LEM_PRIO_ORD_SELCT_CURSTT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162570 entries, 0 to 1162569
Data columns (total 15 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   LOAD_DT                        1162570 non-null  int64  
 1   VIN                            1162570 non-null  object 
 2   RGN                            1162570 non-null  object 
 3   CTPV                           1162570 non-null  object 
 4   DPF_EXTRNS_YN                  1162570 non-null  object 
 5   VHCTY_CD                       1162558 non-null  object 
 6   VHCTY_TY                       1162570 non-null  object 
 7   SEASON_1ODR_CRDN_NOCS          1162570 non-null  float64
 8   SEASON_2ODR_CRDN_NOCS          1162570 non-null  float64
 9   SEASON_3ODR_CRDN_NOCS          1162570 non-null  float64
 10  SEASON_4ODR_CRDN_NOCS          1162570 non-null  float64
 11  SEASON_1ODR_CRDN_NOCS_DY_AVRG  1162570 non-null  float64
 12  SEASON_2ODR_CR

In [ ]:
expdf = STD_BD_GRD4_LEM_PRIO_ORD_SELCT_CURSTT
table_nm = 'STD_BD_GRD4_LEM_PRIO_ORD_SELCT_CURSTT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 5s
we.import_from_pandas(expdf, table_nm)

## STD_BD_GRD5_DS_CRRLTN_CFFCNT

In [ ]:
STD_BD_GRD5_DS_CRRLTN_CFFCNT = pd.read_csv(os.path.join(df3_fold, 'STD_BD_GRD5_DS_CRRLTN_CFFCNT.csv'))
STD_BD_GRD5_DS_CRRLTN_CFFCNT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162570 entries, 0 to 1162569
Data columns (total 15 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   LOAD_DT                        1162570 non-null  int64  
 1   VIN                            1162570 non-null  object 
 2   RGN                            1162570 non-null  object 
 3   CTPV                           1162570 non-null  object 
 4   DPF_EXTRNS_YN                  1162570 non-null  object 
 5   VHCTY_CD                       1162558 non-null  object 
 6   VHCTY_TY                       1162570 non-null  object 
 7   SEASON_1ODR_CRDN_NOCS          1162570 non-null  float64
 8   SEASON_2ODR_CRDN_NOCS          1162570 non-null  float64
 9   SEASON_3ODR_CRDN_NOCS          1162570 non-null  float64
 10  SEASON_4ODR_CRDN_NOCS          1162570 non-null  float64
 11  SEASON_1ODR_CRDN_NOCS_DY_AVRG  1162570 non-null  float64
 12  SEASON_2ODR_CR

In [ ]:
expdf = STD_BD_GRD5_DS_CRRLTN_CFFCNT
table_nm = 'STD_BD_GRD5_DS_CRRLTN_CFFCNT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 5s
we.import_from_pandas(expdf, table_nm)

## STD_BD_GRD5_LEM_PRIO_ORD_SELCT_CURSTT

In [ ]:
STD_BD_GRD5_LEM_PRIO_ORD_SELCT_CURSTT = pd.read_csv(os.path.join(df3_fold, 'STD_BD_GRD5_LEM_PRIO_ORD_SELCT_CURSTT.csv'))
STD_BD_GRD5_LEM_PRIO_ORD_SELCT_CURSTT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162570 entries, 0 to 1162569
Data columns (total 15 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   LOAD_DT                        1162570 non-null  int64  
 1   VIN                            1162570 non-null  object 
 2   RGN                            1162570 non-null  object 
 3   CTPV                           1162570 non-null  object 
 4   DPF_EXTRNS_YN                  1162570 non-null  object 
 5   VHCTY_CD                       1162558 non-null  object 
 6   VHCTY_TY                       1162570 non-null  object 
 7   SEASON_1ODR_CRDN_NOCS          1162570 non-null  float64
 8   SEASON_2ODR_CRDN_NOCS          1162570 non-null  float64
 9   SEASON_3ODR_CRDN_NOCS          1162570 non-null  float64
 10  SEASON_4ODR_CRDN_NOCS          1162570 non-null  float64
 11  SEASON_1ODR_CRDN_NOCS_DY_AVRG  1162570 non-null  float64
 12  SEASON_2ODR_CR

In [ ]:
expdf = STD_BD_GRD5_LEM_PRIO_ORD_SELCT_CURSTT
table_nm = 'STD_BD_GRD5_LEM_PRIO_ORD_SELCT_CURSTT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 5s
we.import_from_pandas(expdf, table_nm)

## STD_BD_DAT_LEM_SELCT

In [26]:
# 0s
STD_BD_DAT_LEM_SELCT = pd.read_csv(os.path.join(df3_fold, 'STD_BD_DAT_LEM_SELCT.csv'))
STD_BD_DAT_LEM_SELCT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396 entries, 0 to 395
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CTPV                 396 non-null    object 
 1   EXHST_GAS_GRD_CD     396 non-null    int64  
 2   VHCTY_CD             396 non-null    object 
 3   VHCTY_TY             396 non-null    object 
 4   SELCT_MKCNT          396 non-null    float64
 5   SENO1_MKCNT          396 non-null    float64
 6   SENO1_RT             396 non-null    float64
 7   SENO2_MKCNT          396 non-null    float64
 8   SENO2_RT             396 non-null    float64
 9   SENO3_MKCNT          396 non-null    float64
 10  SENO3_RT             396 non-null    float64
 11  RDCDVC_EXTRNS_MKCNT  396 non-null    int64  
 12  LOAD_DT              396 non-null    int64  
dtypes: float64(7), int64(3), object(3)
memory usage: 40.3+ KB


In [27]:
expdf = STD_BD_DAT_LEM_SELCT
table_nm = 'STD_BD_DAT_LEM_SELCT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 1s
we.import_from_pandas(expdf, table_nm)

## STD_BD_DAT_SELCT_PNT_CURSTT

In [28]:
# 0s
STD_BD_DAT_SELCT_PNT_CURSTT = pd.read_csv(os.path.join(df3_fold, 'STD_BD_DAT_SELCT_PNT_CURSTT.csv'))
STD_BD_DAT_SELCT_PNT_CURSTT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161 entries, 0 to 160
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   EXHST_GAS_GRD_CD     161 non-null    int64  
 1   SELCT_PNT_RNG        161 non-null    object 
 2   VHCTY_CD             161 non-null    object 
 3   VHCTY_TY             161 non-null    object 
 4   VHCL_MKCNT           161 non-null    int64  
 5   SMO_MSRMT_VAL        161 non-null    float64
 6   DY_AVRG_DRVNG_DSTNC  161 non-null    float64
 7   RCNT_INSP_ELPSD_WHL  161 non-null    float64
 8   RUN_LMT_NOCS         161 non-null    float64
 9   LOAD_DT              161 non-null    int64  
dtypes: float64(4), int64(3), object(3)
memory usage: 12.7+ KB


In [29]:
expdf = STD_BD_DAT_SELCT_PNT_CURSTT
table_nm = 'STD_BD_DAT_SELCT_PNT_CURSTT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 1s
we.import_from_pandas(expdf, table_nm)

# 동양 요청 파일

## STD_BD_CAR_CURSTT_MOD_4RD_INS

In [6]:
# 0s
STD_BD_CAR_CURSTT_MOD_4RD_INS = pd.read_csv(os.path.join(df1_fold, 'STD_BD_CAR_CURSTT_MOD_4RD_INS.csv'), encoding='cp949')
STD_BD_CAR_CURSTT_MOD_4RD_INS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306 entries, 0 to 305
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   YR          306 non-null    int64 
 1   CRTR_YM     306 non-null    int64 
 2   FUEL_CD     306 non-null    object
 3   RGN         306 non-null    object
 4   CTPV_NM     306 non-null    object
 5   VHCL_MKCNT  306 non-null    object
 6   LOAD_DT     306 non-null    object
dtypes: int64(2), object(5)
memory usage: 16.9+ KB


In [7]:
expdf = STD_BD_CAR_CURSTT_MOD_4RD_INS
table_nm = 'STD_BD_CAR_CURSTT_MOD_4RD_INS'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 1s
we.import_from_pandas(expdf, table_nm)

## STD_BD_CAR_CURSTT_MOD2_4RD_INS

In [8]:
# 0s
STD_BD_CAR_CURSTT_MOD2_4RD_INS = pd.read_csv(os.path.join(df1_fold, 'STD_BD_CAR_CURSTT_MOD2_4RD_INS.csv'), encoding='cp949')
STD_BD_CAR_CURSTT_MOD2_4RD_INS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   YR           96 non-null     int64 
 1   CRTR_YM      96 non-null     int64 
 2   VHCTY_CD_NM  96 non-null     object
 3   FUEL_CD      96 non-null     object
 4   VHCL_MKCNT   84 non-null     object
 5   LOAD_DT      96 non-null     object
dtypes: int64(2), object(4)
memory usage: 4.6+ KB


In [9]:
expdf = STD_BD_CAR_CURSTT_MOD2_4RD_INS
table_nm = 'STD_BD_CAR_CURSTT_MOD2_4RD_INS'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 1s
we.import_from_pandas(expdf, table_nm)

# code end